In [61]:
import pandas as pd
import psycopg2
import hashlib

In [62]:
df = pd.read_csv('nba_teams.csv', delimiter=',' )
conn = psycopg2.connect(
    dbname="nba_sql2",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()
SEASON_ID = 1

In [63]:
df['Team']

0              Atlanta Hawks
1             Boston Celtics
2              Brooklyn Nets
3          Charlotte Hornets
4              Chicago Bulls
5        Cleveland Cavaliers
6           Dallas Mavericks
7             Denver Nuggets
8            Detroit Pistons
9      Golden State Warriors
10           Houston Rockets
11            Indiana Pacers
12      Los Angeles Clippers
13        Los Angeles Lakers
14         Memphis Grizzlies
15                Miami Heat
16           Milwaukee Bucks
17    Minnesota Timberwolves
18      New Orleans Pelicans
19           New York Knicks
20     Oklahoma City Thunder
21             Orlando Magic
22       Philadelphia Sixers
23              Phoenix Suns
24    Portland Trail Blazers
25          Sacramento Kings
26         San Antonio Spurs
27           Toronto Raptors
28                 Utah Jazz
29        Washington Wizards
Name: Team, dtype: object

In [67]:
index = 1
for _, row in df.iterrows():

    try:
        # Insert into Player
        cur.execute("""
            INSERT INTO Team (team_id, name)
            VALUES (%s, %s)
        """, (index,row['Team']))

        print(f"✅ Inserted {team_name}")
        index = index+1
        
    except Exception as e:
        print(f"❌ Error inserting {row['Team']}: {e}")

conn.commit()

✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards


In [71]:
df = pd.read_csv('Player_Season_stats.csv', delimiter=';' , encoding="cp1252")

# --- Keep only first entry for each player (skip TOT/MIN duplicates) ---
df = df.drop_duplicates(subset=['Player'], keep='first')

In [72]:
# --- Database connection ---
conn = psycopg2.connect(
    dbname="nba_sql2",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# --- Fixed season ID ---
SEASON_ID = 1

In [77]:
conn.rollback();

In [78]:

for idx,row in df.iterrows():
    player_name = row['firstName'] + ' ' + row['lastName']
    
    cur.execute("SELECT player_id FROM PLAYER WHERE name = %s;", (player_name,))
    player_id = cur.fetchone()
    if player_id is None:
        continue

    game_id = row['gameId']

    pts = row['points']
    trb = row['reboundsTotal']
    ast = row['assists']
    fg_pct = row['fieldGoalsPercentage']
    fg3_pct = row['threePointersPercentage']
    ft_pct = row['freeThrowsPercentage']

    
    try:
        cur.execute("""
            INSERT INTO Player_Game (player_id, game_id, pts, trb, ast, fg_pct, fg3_pct, ft_pct)
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
        """, (player_id, game_id, pts, trb, ast, fg_pct, fg3_pct, ft_pct))

        print(f"✅ Inserted {player_id}")


    except Exception as e:
        print(f"❌ Error: {e}")
        conn.rollback()


✅ Inserted Precious Achiuwa
✅ Inserted Steven Adams
✅ Inserted Bam Adebayo
✅ Inserted Ochai Agbaji
✅ Inserted Santi Aldama
✅ Inserted Nickeil Alexander-Walker
✅ Inserted Grayson Allen
✅ Inserted Jarrett Allen
✅ Inserted Jose Alvarado
✅ Inserted Kyle Anderson
✅ Inserted Giannis Antetokounmpo
✅ Inserted Thanasis Antetokounmpo
✅ Inserted Cole Anthony
✅ Inserted OG Anunoby
...

In [5]:
conn.commit()
cur.close()
conn.close()
#uneseni podaci za svaku tekmu za svakog igraca

In [ ]:
#unosenje Team_Season podataka

In [2]:
import psycopg2
import pandas as pd

In [3]:
DB_CONFIG = {
    "dbname" : "nba_sql2",
    "user" : "postgres",
    "password" : "1234",
    "host" : "localhost",
    "port" : "5432"
}

SEASON_ID = 1

conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()

In [4]:
df = pd.read_csv('team_season.csv', delimiter=',')

In [5]:
for idx,row in df.iterrows():
    team_name = row['Team']

    cur.execute("SELECT team_id FROM TEAM WHERE name = %s;", (team_name,))
    team_id = cur.fetchone()

    wins, losses = row['Overall'].split('-')

    try:
        cur.execute("""
            INSERT INTO Team_Season (team_id, season_id, wins, losses)
            VALUES (%s,%s,%s,%s)
        """, (team_id, SEASON_ID, wins, losses))

        print(f"✅ Inserted {team_name}")


    except Exception as e:
        print(f"❌ Error: {e}")
        conn.rollback()


# team_id , season_id , wins, losses

✅ Inserted Milwaukee Bucks
✅ Inserted Boston Celtics
✅ Inserted Philadelphia 76ers
✅ Inserted Denver Nuggets
✅ Inserted Cleveland Cavaliers
✅ Inserted Memphis Grizzlies
✅ Inserted Sacramento Kings
✅ Inserted New York Knicks
✅ Inserted Brooklyn Nets
✅ Inserted Phoenix Suns
✅ Inserted Golden State Warriors
✅ Inserted Los Angeles Clippers
✅ Inserted Miami Heat
✅ Inserted Los Angeles Lakers
✅ Inserted Minnesota Timberwolves
✅ Inserted New Orleans Pelicans
✅ Inserted Atlanta Hawks
✅ Inserted Toronto Raptors
✅ Inserted Chicago Bulls
✅ Inserted Oklahoma City Thunder
✅ Inserted Dallas Mavericks
✅ Inserted Utah Jazz
✅ Inserted Indiana Pacers
✅ Inserted Washington Wizards
✅ Inserted Orlando Magic
✅ Inserted Portland Trail Blazers
✅ Inserted Charlotte Hornets
✅ Inserted Houston Rockets
✅ Inserted San Antonio Spurs
✅ Inserted Detroit Pistons


In [6]:
conn.commit()
cur.close()
conn.close()